# 🕒 Média de passageiros por hora — Maio (toda a frota)

Esta consulta cria a *view* `DateTimeMediumPassenger` que calcula a **média de passageiros por hora do dia**
para o **mês de maio** (`Mes_Inicio_Viagem = 5` e `Mes_Fim_Viagem = 5`), considerando **todos os táxis da frota**.


In [0]:
%sql
-- Contexto (catálogo e schema) onde a view será criada
USE CATALOG ifood_case;
USE SCHEMA  nytaxi;

-- Cria (ou substitui) a VIEW que consolida a média de passageiros por hora do dia em maio
CREATE OR REPLACE VIEW DateTimeMediumPassenger AS

-- =========================================
-- CTE: prepara dados do mês de maio
-- - Deriva a data no formato dd-MM-yyyy (Data_Viagem)
-- - Extrai a hora do dia (Hora_Dia) a partir de 'Hora_Inicio_Viagem'
-- - Converte 'passenger_count' para DOUBLE (para AVG)
-- - Mantém colunas úteis para auditoria/extensões (vendorId, total_amount, timestamps)
-- - Filtra apenas corridas cujo início e fim estão em maio (Mes_Inicio_Viagem = Mes_Fim_Viagem = 5)
--   >> Observação: o nome da CTE coincide com o nome da view; é válido, mas se preferir clareza, renomeie a CTE.
-- =========================================
WITH DateTimeMediumPassenger AS (
  SELECT
      DATE_FORMAT(tpep_pickup_datetime, 'dd-MM-yyyy')         AS Data_Viagem     -- data do pickup (dia/mês/ano)
    , TRY_CAST(SUBSTRING(Hora_Inicio_Viagem, 1, 2) AS INT)    AS Hora_Dia        -- hora (0–23) extraída do campo textual
    , TRY_CAST(passenger_count AS DOUBLE)                     AS passenger_count -- casting para permitir AVG
    , vendorId                                                                   -- mantido para eventuais recortes por fornecedor
    , total_amount                                                               -- mantido para futuras análises de receita
    , tpep_dropoff_datetime                                                      -- timestamp de término
    , tpep_pickup_datetime                                                       -- timestamp de início
  FROM GoldYellow
  WHERE
    Mes_Fim_Viagem        = 5                                                   -- fim em maio
    AND Mes_Inicio_Viagem = 5                                                   -- início em maio
)

-- =========================================
-- Agregação final: média por Data_Viagem + Hora_Dia
-- - Responde diretamente à pergunta solicitada
-- - Filtra horas válidas (0–23)
-- - Calcula AVG(passenger_count) com 2 casas decimais
-- =========================================
SELECT
    Data_Viagem
  , Hora_Dia
  , ROUND(AVG(passenger_count), 2) AS Media_Passageiros
FROM DateTimeMediumPassenger
WHERE Hora_Dia BETWEEN 0 AND 23
GROUP BY
    Data_Viagem
  , Hora_Dia
ORDER BY
    Data_Viagem ASC
  , Hora_Dia   ASC;


In [0]:
%sql
-- Contexto (catálogo e schema) onde a view será criada
USE CATALOG ifood_case;
USE SCHEMA  nytaxi;

SELECT * FROM DateTimeMediumPassenger